In [1]:
import pandas as pd

# Load dataset
data_path = r"C:\Users\sharl\Desktop\Final year project\Fake Job System\Data\Raw\fake_job_postings.csv"
df = pd.read_csv(data_path)

# Basic inspection
df.head()


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [3]:
df.shape


(17880, 18)

In [5]:
df.columns


Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [7]:
df['fraudulent'].value_counts()


fraudulent
0    17014
1      866
Name: count, dtype: int64

In [9]:
df[['title', 'description', 'fraudulent']].sample(5)


,title,description,fraudulent
2327,Batley Business Admin Apprenticeship Governmen...,Government funding is only available for 16-18...,0
337,Customer Service Associate,The Customer Service Associate will be based i...,0
9208,Forklift Operator,Qualifications6 months of warehouse experience...,0
14565,Android Developer - Official London Travel App,ABOUT What Now?! is an innovative new app that...,0
10468,Production Supervisor - Production Supervisory...,Experienced Production Supervisor is required ...,0


In [11]:
df['fraudulent'].value_counts(normalize=True)


fraudulent
0    0.951566
1    0.048434
Name: proportion, dtype: float64

In [13]:
text_columns = [
    'title',
    'company_profile',
    'description',
    'requirements',
    'benefits'
]

target = 'fraudulent'

df_selected = df[text_columns + [target]].copy()
df_selected.head()


,title,company_profile,description,requirements,benefits,fraudulent
0,Marketing Intern,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0
1,Customer Service - Cloud Video Production,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0
2,Commissioning Machinery Assistant (CMA),Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0
3,Account Executive - Washington DC,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0
4,Bill Review Manager,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0


In [15]:
df_selected.loc[:, text_columns] = df_selected[text_columns].fillna("")


In [17]:
df_selected.loc[:, 'text'] = (
    df_selected['title'] + " " +
    df_selected['company_profile'] + " " +
    df_selected['description'] + " " +
    df_selected['requirements'] + " " +
    df_selected['benefits']
)
df_selected.isnull().sum()


title              0
company_profile    0
description        0
requirements       0
benefits           0
fraudulent         0
text               0
dtype: int64

In [19]:
df_selected[['text', 'fraudulent']].sample(3)


,text,fraudulent
14642,iOS Developer ehumanlife is a medical platform...,0
12653,Manufacturing Representative - HSES Manufactur...,0
1685,Project Manager Valor Services provides Workfo...,0


In [21]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)   # remove URLs
    text = re.sub(r"[^a-z\s]", " ", text)        # remove punctuation & numbers
    text = re.sub(r"\s+", " ", text).strip()     # normalize spaces
    return text

df_selected['clean_text'] = df_selected['text'].apply(clean_text)


In [22]:
df_selected[['text', 'clean_text']].sample(2)


,text,clean_text
9120,Commercial Real Estate Paralegal- Atlanta-Buck...,commercial real estate paralegal atlanta buckh...
16623,"Appointment Setter avg. 1,000-1,100 weekly Plu...",appointment setter avg weekly plus commission ...


In [25]:
processed_path = r"C:\Users\sharl\Desktop\Final year project\Fake Job System\Data\Processed\clean_jobs.csv"
df_selected[['clean_text', 'fraudulent']].to_csv(processed_path, index=False)


In [27]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\sharl\Desktop\Final year project\Fake Job System\Data\Processed\clean_jobs.csv"
)

df['fraudulent'].value_counts()


fraudulent
0    17014
1      866
Name: count, dtype: int64

In [29]:
X = df['clean_text']
y = df['fraudulent']


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [34]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)

model.fit(X_train_tfidf, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [37]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test_tfidf)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[3320   83]
 [  17  156]]
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      3403
           1       0.65      0.90      0.76       173

    accuracy                           0.97      3576
   macro avg       0.82      0.94      0.87      3576
weighted avg       0.98      0.97      0.97      3576



In [39]:
import joblib

joblib.dump(model, r"C:\Users\sharl\Desktop\Final year project\Fake Job System\Model\fraud_model.joblib")
joblib.dump(tfidf, r"C:\Users\sharl\Desktop\Final year project\Fake Job System\Model\tfidf_vectorizer.joblib")


['C:\\Users\\sharl\\Desktop\\Final year project\\Fake Job System\\Model\\tfidf_vectorizer.joblib']